In [1]:
from datetime import timedelta, datetime
import rsa_headers as rh
import rsa_data as rd
import main
files = r"C:\Users\MB2705851\Downloads\SMEC RSA Files_Oct&Nov2021\0400_211115103640.RSA"
import pandas as pd
pd.set_option("display.max_columns", None)

testfile = r"C:\Users\MB2705851\Desktop\Temp\test.csv"

In [2]:
df = main.to_df(files)

In [3]:
header = rh.Headers(df)
header = header.header

In [ ]:
print(header)

In [ ]:
data = df.loc[(df[0] == "30") & (df[1].isin(["0", "1", "2", "3", "4"]))].dropna(
    axis=1, how="all"
)
dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
    axis=1, how="all"
)

In [ ]:
data[1].isin(["0","2"]).any()
# if data[1].isin(["0","2"]).any(axis=1):
#     print(data[1])
# else:
#     print('fix')


In [ ]:
def dtype30(df: pd.DataFrame) -> pd.DataFrame:
    data = df.loc[(df[0] == "30") & (df[1].isin(["0", "1", "2", "3", "4"]))].dropna(
        axis=1, how="all"
    )
    dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
        axis=1, how="all"
    )
    if data.empty:
        pass
    else:
        if data[1].all() == "1":
            ddf = data.iloc[:, 3:]
            ddf = pd.DataFrame(ddf).dropna(axis=1, how="all")
            ddf.columns = [
                "end_datetime",
                "end_time",
                "duration_min",
                "lane_number",
                "unknown_vehicle_error_class",
                "motorcycle",
                "light_motor_vehicles",
                "light_motor_vehicles_towing",
                "two_axle_busses",
                "two_axle_6_tyre_single_units",
                "busses_with_3_or_4_axles",
                "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
                "three_axle_single_unit_including_single_axle_light_trailer",
                "four_or_less_axle_including_a_single_trailer",
                "buses_with_5_or_more_axles",
                "three_axle_single_unit_and_light_trailer_more_than_4_axles",
                "five_axle_single_trailer",
                "six_axle_single_trailer",
                "five_or_less_axle_multi_trailer",
                "six_axle_multi_trailer",
                "seven_axle_multi_trailer",
                "eight_or_more_axle_multi_trailer",
            ]
            ddf["heavy_vehicle"] = 0

        elif data[1].isin(["0","2"]).any():
            ddf = data.iloc[:, 2:]
            ddf = pd.DataFrame(ddf).dropna(axis=1, how="all")
            if ddf.shape[1] == 7:
                ddf.columns = [
                    "end_datetime",
                    "end_time",
                    "duration_min",
                    "lane_number",
                    "unknown_vehicle_error_class",
                    "light_motor_vehicles",
                    "heavy_vehicle",
                ]
                ddf = pd.concat(
                    [
                        ddf,
                        pd.DataFrame(
                            columns=[
                                "motorcycle",
                                "light_motor_vehicles",
                                "light_motor_vehicles_towing",
                                "two_axle_busses",
                                "two_axle_6_tyre_single_units",
                                "busses_with_3_or_4_axles",
                                "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
                                "three_axle_single_unit_including_single_axle_light_trailer",
                                "four_or_less_axle_including_a_single_trailer",
                                "buses_with_5_or_more_axles",
                                "three_axle_single_unit_and_light_trailer_more_than_4_axles",
                                "five_axle_single_trailer",
                                "six_axle_single_trailer",
                                "five_or_less_axle_multi_trailer",
                                "six_axle_multi_trailer",
                                "seven_axle_multi_trailer",
                                "eight_or_more_axle_multi_trailer",
                            ]
                        ),
                    ]
                )
            else:
                ddf.columns = [
                    "end_datetime",
                    "end_time",
                    "duration_min",
                    "lane_number",
                    "unknown_vehicle_error_class",
                    "motorcycle",
                    "light_motor_vehicles",
                    "light_motor_vehicles_towing",
                    "two_axle_busses",
                    "two_axle_6_tyre_single_units",
                    "busses_with_3_or_4_axles",
                    "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
                    "three_axle_single_unit_including_single_axle_light_trailer",
                    "four_or_less_axle_including_a_single_trailer",
                    "buses_with_5_or_more_axles",
                    "three_axle_single_unit_and_light_trailer_more_than_4_axles",
                    "five_axle_single_trailer",
                    "six_axle_single_trailer",
                    "five_or_less_axle_multi_trailer",
                    "six_axle_multi_trailer",
                    "seven_axle_multi_trailer",
                    "eight_or_more_axle_multi_trailer",
                ]

        ddf = ddf.dropna(
            axis=0,
            how="all",
            subset=[
                "unknown_vehicle_error_class",
                "light_motor_vehicles",
                "motorcycle",
                "light_motor_vehicles",
                "light_motor_vehicles_towing",
                "two_axle_busses",
                "two_axle_6_tyre_single_units",
                "busses_with_3_or_4_axles",
                "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
                "three_axle_single_unit_including_single_axle_light_trailer",
                "four_or_less_axle_including_a_single_trailer",
                "buses_with_5_or_more_axles",
                "three_axle_single_unit_and_light_trailer_more_than_4_axles",
                "five_axle_single_trailer",
                "six_axle_single_trailer",
                "five_or_less_axle_multi_trailer",
                "six_axle_multi_trailer",
                "seven_axle_multi_trailer",
                "eight_or_more_axle_multi_trailer",
            ],
        )

        # ddf=ddf.fillna(0)
        ddf["duration_min"] = ddf["duration_min"].astype(int)
        ddf["lane_number"] = ddf["lane_number"].astype(int)
        ddf["unknown_vehicle_error_class"] = ddf[
            "unknown_vehicle_error_class"
        ].astype(int)
        ddf["light_motor_vehicles"] = ddf["light_motor_vehicles"].astype(int)
        ddf["light_motor_vehicles_towing"] = ddf[
            "light_motor_vehicles_towing"
        ].astype(int)
        ddf["two_axle_busses"] = ddf["two_axle_busses"].astype(int)
        ddf["two_axle_6_tyre_single_units"] = ddf[
            "two_axle_6_tyre_single_units"
        ].astype(int)
        ddf["busses_with_3_or_4_axles"] = ddf["busses_with_3_or_4_axles"].astype(
            int
        )
        ddf["two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"] = ddf[
            "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"
        ].astype(int)
        ddf["three_axle_single_unit_including_single_axle_light_trailer"] = ddf[
            "three_axle_single_unit_including_single_axle_light_trailer"
        ].astype(int)
        ddf["four_or_less_axle_including_a_single_trailer"] = ddf[
            "four_or_less_axle_including_a_single_trailer"
        ].astype(int)
        ddf["buses_with_5_or_more_axles"] = ddf[
            "buses_with_5_or_more_axles"
        ].astype(int)
        ddf["three_axle_single_unit_and_light_trailer_more_than_4_axles"] = ddf[
            "three_axle_single_unit_and_light_trailer_more_than_4_axles"
        ].astype(int)
        ddf["five_axle_single_trailer"] = ddf["five_axle_single_trailer"].astype(
            int
        )
        ddf["six_axle_single_trailer"] = ddf["six_axle_single_trailer"].astype(int)
        ddf["five_or_less_axle_multi_trailer"] = ddf[
            "five_or_less_axle_multi_trailer"
        ].astype(int)
        ddf["six_axle_multi_trailer"] = ddf["six_axle_multi_trailer"].astype(int)
        ddf["seven_axle_multi_trailer"] = ddf["seven_axle_multi_trailer"].astype(
            int
        )
        ddf["eight_or_more_axle_multi_trailer"] = ddf[
            "eight_or_more_axle_multi_trailer"
        ].astype(int)

        ddf["total_light_vehicles_type30"] = (
            ddf["motorcycle"].astype(int)
            + ddf["light_motor_vehicles"].astype(int)
            + ddf["light_motor_vehicles_towing"].astype(int)
        )
        ddf["total_heavy_vehicles_type30"] = (
            ddf["two_axle_busses"].astype(int)
            + ddf["two_axle_6_tyre_single_units"].astype(int)
            + ddf["busses_with_3_or_4_axles"].astype(int)
            + ddf[
                "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"
            ].astype(int)
            + ddf[
                "three_axle_single_unit_including_single_axle_light_trailer"
            ].astype(int)
            + ddf["four_or_less_axle_including_a_single_trailer"].astype(int)
            + ddf["buses_with_5_or_more_axles"].astype(int)
            + ddf[
                "three_axle_single_unit_and_light_trailer_more_than_4_axles"
            ].astype(int)
            + ddf["five_axle_single_trailer"].astype(int)
            + ddf["six_axle_single_trailer"].astype(int)
            + ddf["five_or_less_axle_multi_trailer"].astype(int)
            + ddf["six_axle_multi_trailer"].astype(int)
            + ddf["seven_axle_multi_trailer"].astype(int)
            + ddf["eight_or_more_axle_multi_trailer"].astype(int)
        )

        ddf["total_vehicles_type30"] = (
            ddf["motorcycle"].astype(int)
            + ddf["light_motor_vehicles"].astype(int)
            + ddf["light_motor_vehicles_towing"].astype(int)
            + ddf["two_axle_busses"].astype(int)
            + ddf["two_axle_6_tyre_single_units"].astype(int)
            + ddf["busses_with_3_or_4_axles"].astype(int)
            + ddf[
                "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"
            ].astype(int)
            + ddf[
                "three_axle_single_unit_including_single_axle_light_trailer"
            ].astype(int)
            + ddf["four_or_less_axle_including_a_single_trailer"].astype(int)
            + ddf["buses_with_5_or_more_axles"].astype(int)
            + ddf[
                "three_axle_single_unit_and_light_trailer_more_than_4_axles"
            ].astype(int)
            + ddf["five_axle_single_trailer"].astype(int)
            + ddf["six_axle_single_trailer"].astype(int)
            + ddf["five_or_less_axle_multi_trailer"].astype(int)
            + ddf["six_axle_multi_trailer"].astype(int)
            + ddf["seven_axle_multi_trailer"].astype(int)
            + ddf["eight_or_more_axle_multi_trailer"].astype(int)
        )

        max_lanes = ddf["lane_number"].max()
        ddf["direction"] = ddf.apply(
            lambda x: "P" if x["lane_number"] <= (int(max_lanes) / 2) else "N",
            axis=1,
        )
        direction = dfh2.loc[dfh2[0] == "L1", 1:3].astype(int)
        direction = direction.drop_duplicates()
        try:
            ddf["forward_direction_code"] = ddf.apply(
                lambda x: Data.get_direction(x["lane_number"], direction), axis=1
            )
            # FIXME: ddf['lane_position_code']=ddf.apply(lambda x: Data.get_lane_position(x['lane_number'],direction),axis=1)
        except Exception:
            ddf["forward_direction_code"] = None
            # ddf['lane_position_code']=None

        ddf["end_datetime"] = ddf.apply(
            lambda x: pd.to_datetime(
                x["end_datetime"] + x["end_time"], format="%y%m%d%H%M"
            )
            if (
                x["end_time"] != "2400"
                and len(str(x["end_datetime"])) == 6
                and len(x["end_time"]) == 4
            )
            else (
                pd.to_datetime(
                    x["end_datetime"] + x["end_time"], format="%Y%m%d%H%M"
                )
                if (
                    x["end_time"] != "2400"
                    and len(str(x["end_datetime"])) == 8
                    and len(x["end_time"]) == 4
                )
                else (
                    pd.to_datetime(x["end_datetime"], format="%y%m%d")
                    + timedelta(days=1)
                    if (
                        x["end_time"] == "2400"
                        and len(str(x["end_datetime"])) == 6
                        and len(x["end_time"]) == 4
                    )
                    else (
                        pd.to_datetime(x["end_datetime"], format="%Y%m%d")
                        + timedelta(days=1)
                        if (
                            x["end_time"] != "2400"
                            and len(str(x["end_datetime"])) == 8
                            and len(x["end_time"]) == 4
                        )
                        else pd.to_datetime(x["end_datetime"])
                    )
                )
            ),
            axis=1,
        )

        ddf = ddf.drop(["end_time"], axis=1)

        ddf["start_datetime"] = pd.to_datetime(
            ddf["end_datetime"]
        ) - pd.to_timedelta(ddf["duration_min"].astype(int), unit="m")
        # ddf['year'] = ddf['start_datetime'].dt.year
        t1 = dfh2.loc[dfh2[0] == "S0", 1].unique()
        ddf["site_id"] = str(t1[0])

        ddf = ddf.drop_duplicates()

        return ddf

In [ ]:
test = dtype30(df)

In [ ]:
print(test)

In [4]:
DATA = rd.Data(df)

In [ ]:
data = DATA.dtype21
data = main.data_join(data, header)
data.drop("station_name", axis=1, inplace=True)

d2 = DATA.dtype30
try:
    d2 = main.data_join(d2, header)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )
except Exception:
    pass

d2 = DATA.dtype70
try:
    data = main.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data['start_datetime'] = data['start_datetime'].astype('datetime64[ns]')
    d2['start_datetime'] = d2['start_datetime'].astype('datetime64[ns]')
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )
except Exception:
    pass

# d2 = DATA.dtype10
# try:
#     data = main.data_join(d2, header)
#     data.drop("station_name", axis=1, inplace=True)
#     data = data.merge(
#         d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
#     )
# except Exception:
#     pass

d2 = DATA.dtype60
try:
    data = main.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )
except Exception:
    pass

In [5]:
data = DATA.dtype21
data = main.data_join(data, header)
data.drop("station_name", axis=1, inplace=True)

d2 = DATA.dtype30
if d2 is None:
    pass
else:
    d2 = main.data_join(d2, header)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )

d2 = DATA.dtype70
if d2 is None:
    pass
else:
    data = main.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data['start_datetime'] = data['start_datetime'].astype('datetime64[ns]')
    d2['start_datetime'] = d2['start_datetime'].astype('datetime64[ns]')
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )

d2 = DATA.dtype10
if d2 is None:
    pass
else:
    data = main.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )


d2 = DATA.dtype60
if d2 is None:
    pass
else:
    data = main.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )


In [ ]:

data2 = main.data_join(data, header.header)
data.drop('station_name', axis=1, inplace=True)
print(data2)

In [6]:
print(data)

                                 header_id              end_datetime_x  \
0     f30ccbfb-3d11-4110-95fd-16e238594c9d  2021-10-21 12:30:00.000000   
1     f30ccbfb-3d11-4110-95fd-16e238594c9d  2021-10-21 12:30:00.000000   
2     f30ccbfb-3d11-4110-95fd-16e238594c9d  2021-10-21 12:30:00.000000   
3     f30ccbfb-3d11-4110-95fd-16e238594c9d  2021-10-21 12:30:00.000000   
4     f30ccbfb-3d11-4110-95fd-16e238594c9d  2021-10-21 12:45:00.000000   
...                                    ...                         ...   
9567                                   NaN                         NaN   
9568                                   NaN                         NaN   
9569                                   NaN                         NaN   
9570                                   NaN                         NaN   
9571                                   NaN                         NaN   

      duration_min_x  lane_number  total_free_flowing_light_vehicles_x  \
0               15.0            1    

In [73]:
data.to_csv(testfile)

In [ ]:
main.push_to_db(
            data,
            "electronic_count_data_partitioned",
            ["site_id", "start_datetime", "lane_number"],
        )